In [21]:
from PIL import Image
from pytesseract import pytesseract
import re

In [13]:
# Defining paths to tesseract.exe
# and the image we would be using
# path_to_tesseract = r"/usr/local/Cellar/tesseract/4.1.3/bin/tesseract"
path_to_tesseract='/home/mateo/code/Matbeac/FritzWebInterface/tesseract'
image_path = r"../MS-Online-Menu-Mains-January.jpg"
## Opening the image & storing it in an image object
img = Image.open(image_path)
  
# Providing the tesseract executable
# location to pytesseract library
# pytesseract.tesseract_cmd = path_to_tesseract
  
# Passing the image object to image_to_string() function
# This function will extract the text from the image
text = pytesseract.image_to_string(img)
  
# Displaying the extracted text
text=text[:-1]

In [18]:
type(text)

str

In [14]:
import pandas as pd
import numpy as np

In [15]:
string_list=[x for x in text.split("\n") if len(x)!=0]

In [16]:
string_list

['[ILESTONE',
 'CHEF PATRICK BOND AND TEAM HAVE RETURNED TO THE MILESTONE’S ROOTS WITH THIS SEASONAL, FROM ~ SCRATCH MENU.',
 'A RANGE OF ROTATING SPECIALS ARE ALSO DISPLAYED ON BLACKBOARDS IN THE RESTAURANT.',
 'STARTERS',
 'Jerusalem Artichoke Soup**',
 'Garlic Oil, Chives & Milestone Bread(GF,V) 5',
 'Black Pudding & Pigs Head Fritter',
 'Fried Hens Egg, DevilSauce(DF) 6.5',
 'Beetroot & House Ricotta',
 'Pickled Beetroot, Pecan Crumb, Linseed Cracker (V) 6.5',
 'Fried Tofu**',
 'Coconut Katsu, Pickled Carrots, Garlic Shoots (VG) 6',
 'MAINS',
 'Denver Steak',
 'Stout Braised Lentils, Celeriac Fondant, Charred Chicory, Caper & Thyme Jus (GF)',
 '16',
 'Pan Fried Bream',
 'Dill Cream Sauce, Samphire, Confit Potato, Garlic Oil (GF)',
 '13',
 'Roast Fennel**',
 'Braised Harissa Chickpeas, Smoked Chickpea & Tahini "Mayo" & Pickle Fennel Salad (VG, GF)',
 '12',
 'Bangers & Mash**',
 'Pork & Leek Sausages, Butter Mash Potato, Cider Jus',
 '12',
 'Mussels @ Clams',
 'Lemongrass Broth, Cori

In [19]:
ingredient_file_path='../Emission_computing/final_ingredients_emissions.csv'
df=pd.read_csv(ingredient_file_path,error_bad_lines=False)

/home/mateo/.pyenv/versions/3.8.12/envs/FritzWebInterface/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [26]:
string_list

['[ILESTONE',
 'CHEF PATRICK BOND AND TEAM HAVE RETURNED TO THE MILESTONE’S ROOTS WITH THIS SEASONAL, FROM ~ SCRATCH MENU.',
 'A RANGE OF ROTATING SPECIALS ARE ALSO DISPLAYED ON BLACKBOARDS IN THE RESTAURANT.',
 'STARTERS',
 'Jerusalem Artichoke Soup**',
 'Garlic Oil, Chives & Milestone Bread(GF,V) 5',
 'Black Pudding & Pigs Head Fritter',
 'Fried Hens Egg, DevilSauce(DF) 6.5',
 'Beetroot & House Ricotta',
 'Pickled Beetroot, Pecan Crumb, Linseed Cracker (V) 6.5',
 'Fried Tofu**',
 'Coconut Katsu, Pickled Carrots, Garlic Shoots (VG) 6',
 'MAINS',
 'Denver Steak',
 'Stout Braised Lentils, Celeriac Fondant, Charred Chicory, Caper & Thyme Jus (GF)',
 '16',
 'Pan Fried Bream',
 'Dill Cream Sauce, Samphire, Confit Potato, Garlic Oil (GF)',
 '13',
 'Roast Fennel**',
 'Braised Harissa Chickpeas, Smoked Chickpea & Tahini "Mayo" & Pickle Fennel Salad (VG, GF)',
 '12',
 'Bangers & Mash**',
 'Pork & Leek Sausages, Butter Mash Potato, Cider Jus',
 '12',
 'Mussels @ Clams',
 'Lemongrass Broth, Cori

In [34]:
for line in string_list:
    line=re.sub('[^A-Za-z0-9]+', ' ', line)
    line=re.sub(r'[0-9]+', '', line)
    ingredient_words=re.split('\s+', line)
    print(ingredient_words)

['', 'ILESTONE']
['CHEF', 'PATRICK', 'BOND', 'AND', 'TEAM', 'HAVE', 'RETURNED', 'TO', 'THE', 'MILESTONE', 'S', 'ROOTS', 'WITH', 'THIS', 'SEASONAL', 'FROM', 'SCRATCH', 'MENU', '']
['A', 'RANGE', 'OF', 'ROTATING', 'SPECIALS', 'ARE', 'ALSO', 'DISPLAYED', 'ON', 'BLACKBOARDS', 'IN', 'THE', 'RESTAURANT', '']
['STARTERS']
['Jerusalem', 'Artichoke', 'Soup', '']
['Garlic', 'Oil', 'Chives', 'Milestone', 'Bread', 'GF', 'V', '']
['Black', 'Pudding', 'Pigs', 'Head', 'Fritter']
['Fried', 'Hens', 'Egg', 'DevilSauce', 'DF', '']
['Beetroot', 'House', 'Ricotta']
['Pickled', 'Beetroot', 'Pecan', 'Crumb', 'Linseed', 'Cracker', 'V', '']
['Fried', 'Tofu', '']
['Coconut', 'Katsu', 'Pickled', 'Carrots', 'Garlic', 'Shoots', 'VG', '']
['MAINS']
['Denver', 'Steak']
['Stout', 'Braised', 'Lentils', 'Celeriac', 'Fondant', 'Charred', 'Chicory', 'Caper', 'Thyme', 'Jus', 'GF', '']
['']
['Pan', 'Fried', 'Bream']
['Dill', 'Cream', 'Sauce', 'Samphire', 'Confit', 'Potato', 'Garlic', 'Oil', 'GF', '']
['']
['Roast', 'Fennel

In [33]:
ingredient_words

['GF',
 'GLUTEN',
 'FREE',
 'V',
 'VEGETARIAN',
 'VG',
 'VEGAN',
 'DF',
 'DAIRY',
 'FREE']

In [43]:
final_dict={'ingredient_parsed':[],"ingredient_from_df":[], 'emission':[]}

for line in string_list:
    line=line.lower()
    line=re.sub('[^A-Za-z0-9]+', ' ', line)
    line=re.sub(r'[0-9]+', '', line)
    ingredient_words=re.split('\s+', line)
    for ingredient in ingredient_words:
        if df[df['ingredient'].str.match(r''+str(ingredient)+'$')==True].ingredient.values.size>0:
            final_dict['ingredient_from_df'].append(df[df['ingredient'].str.match(r''+str(ingredient)+'$')== True].ingredient.iloc[0])
            final_dict['emission'].append(df[df['ingredient'].str.match(r''+str(ingredient)+'$')== True].emissions.iloc[0])
            final_dict["ingredient_parsed"].append(line)

In [44]:
final_df=pd.DataFrame(final_dict)

In [45]:
final_df

,ingredient_parsed,ingredient_from_df,emission
0,jerusalem artichoke soup,artichoke,338.0
1,garlic oil chives milestone bread gf v,garlic,452.0
2,garlic oil chives milestone bread gf v,chives,1133.0
3,garlic oil chives milestone bread gf v,bread,1215.0
4,fried hens egg devilsauce df,egg,4506.0
5,beetroot house ricotta,beetroot,1136.0
6,beetroot house ricotta,ricotta,6440.0
7,pickled beetroot pecan crumb linseed cracker v,beetroot,1136.0
8,pickled beetroot pecan crumb linseed cracker v,pecan,2129.0
9,fried tofu,tofu,3160.0


In [46]:
final_df['ingredient_parsed']=final_df['ingredient_parsed'].astype(str)

In [47]:
result=final_df.groupby('ingredient_parsed').sum()

In [63]:
result=result.sort_values(by="emission",ascending=True)

In [105]:
df=result[result['emission']==result['emission'].min()].iloc[0,0]

In [106]:
df

338.0